## IMDB数据集导入

In [1]:
import keras
from keras.datasets import imdb
(train_data,train_labels),(test_data,test_labels)= imdb.load_data(num_words=10000)

Using TensorFlow backend.


num_words用来控制向量的长度，过滤低频词汇

In [6]:
#查看数据集形状
print("训练集：",train_data.shape)
print("测试集：",test_data.shape)

训练集： (25000,)
测试集： (25000,)


In [14]:
#查看标签
print(train_labels[:10])

[1 0 0 1 0 0 1 0 1 0]


In [13]:
#查看两条训练数据长度
print("第一个训练数据长度：",len(train_data[1]))
print("第二个训练数据长度：",len(train_data[2]))

第一个训练数据长度： 189
第二个训练数据长度： 141


我们可以看出样本之间的长度是不一样的，因此我们需要对其进行处理，处理这种长度不一样的文本向量，一般有两种方式，第一种是Embedding，第二种是one-hot

## one-hot方法将样本维度变成一致

In [2]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results= np.zeros([len(sequences),dimension])
    for i, sequence in enumerate(sequences):
        results[i,sequence]=1#逐行按照将对应的Index设置为1
    return results
x_train= vectorize_sequences(train_data)
x_test= vectorize_sequences(test_data)

内存爆炸！！！

In [3]:
print("one-hot后训练集维度：",x_train.shape)
print("one-hot后测试集维度：",x_test.shape)

one-hot后训练集维度： (25000, 10000)
one-hot后测试集维度： (25000, 10000)


In [31]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


经过One-Hot后数据维度变成一致，可以用于训练了，但是维度还是太大了，有10000，有必要的化我觉得可以降维

## 搭建一个简单的神经网络来训练

In [10]:
from keras.layers import Dense
from keras import models
def model():
    model= models.Sequential()
    model.add(Dense(128,input_shape=(10000,),activation="relu"))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss="binary_crossentropy",metrics=['acc'],optimizer="rmsprop")
    return model

In [11]:
model= model()

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               1280128   
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 1,288,449
Trainable params: 1,288,449
Non-trainable params: 0
_________________________________________________________________


In [14]:
history= model.fit(x_train, train_labels, epochs=5,batch_size=512)

Epoch 1/5
25000/25000 [==============================] - 9s 375us/step - loss: 0.4180 - acc: 0.8217 2s - loss: 0.4441 -
Epoch 2/5
25000/25000 [==============================] - 4s 168us/step - loss: 0.2345 - acc: 0.9088
Epoch 3/5
25000/25000 [==============================] - 4s 144us/step - loss: 0.1754 - acc: 0.9333
Epoch 4/5
25000/25000 [==============================] - 4s 142us/step - loss: 0.1245 - acc: 0.9546
Epoch 5/5
25000/25000 [==============================] - 4s 142us/step - loss: 0.0828 - acc: 0.9715


In [15]:
model.evaluate(x_test,test_labels)

25000/25000 [==============================] - 10s 411us/step


[0.37436788031578067, 0.87763999999999998]

model.evaluate返回一个列表，列表的第一个元素代表损失，第二个元素代表你的指标

训练完之后内存暴增

整理几个问题：  
1. input_data和input_dim有什么区别？该怎么设置？
2. 每次训练完内存都暴涨，怎么自动清除无用内存？
3. 模型的超参数该怎么设置？比如网络的深度、每层神经元个数？